In [3]:
import pandas as pd
import glob
from read_csv_gz import read_csv_gz
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
ed_diagnosis_df = read_csv_gz("ed_data/diagnosis.csv.gz")
ed_edstays_df = read_csv_gz("ed_data/edstays.csv.gz")
ed_medrecon_df = read_csv_gz("ed_data/medrecon.csv.gz")
ed_pyxis_df = read_csv_gz("ed_data/pyxis.csv.gz")
ed_triage_df = read_csv_gz("ed_data/triage.csv.gz")
ed_vitalsign_df = read_csv_gz("ed_data/vitalsign.csv.gz")
hosp_admissions_df = read_csv_gz("hosp_data/admissions.csv.gz")
hosp_patients_df = read_csv_gz("hosp_data/patients.csv.gz")

In [19]:
# ed_edstays_df.head(5)
ed_edstays_df.describe()

,subject_id,hadm_id,stay_id
count,4.250870e+05,2.030160e+05,4.250870e+05
mean,1.500871e+07,2.499495e+07,3.499735e+07
std,2.878486e+06,2.888745e+06,2.888342e+06
min,1.000003e+07,2.000002e+07,3.000001e+07
25%,1.251789e+07,2.248982e+07,3.249799e+07
50%,1.501628e+07,2.499935e+07,3.499395e+07
75%,1.749900e+07,2.749388e+07,3.750387e+07
max,1.999999e+07,2.999981e+07,3.999996e+07


In [6]:
# hosp_admissions_df.head(15)

In [7]:
# Why does hosptial admittime come BEFORE edouttime in most cases?
(hosp_admissions_df['edouttime'] < hosp_admissions_df['admittime']).value_counts()

False    537006
True       9022
Name: count, dtype: int64

In [8]:
# hosp_patients_df.head(10)

In [9]:
def merge_df(df_chef, df_2, df_3):
    """
    Merge the three dataframes together and calculate the patient age at the adimission time to emergency department.
    :param df_chef: the main dataframe, edstays
    :param df_2: the second dataframe, patients, which contains the patient's anchor age and anchor year
    :param df_3: the third dataframe, admissions, which contains the patient's language (Note: insurance and marital status could be different for a given patient thus are not used in this function)
    :return: a new dataframe with the patient's age and language at the admission time to emergency department
    """ 
    df_2_cleaned = df_2[['subject_id', 'anchor_age', 'anchor_year']].drop_duplicates()
    # df_3_cleaned = df_3[['subject_id', 'insurance', 'marital_status', 'language']].drop_duplicates(subset='subject_id')
    df_3_cleaned = df_3[['subject_id', 'language']].drop_duplicates() 
    ans = pd.merge(df_chef, df_3_cleaned, on='subject_id', how='left') 
    ans = pd.merge(ans, df_2_cleaned, on='subject_id', how='left') 
    ans['admission_age'] = pd.to_datetime(ans['intime']).dt.year - ans['anchor_year'] + ans['anchor_age'] # calculate age at admission time
    ans = ans.drop(columns=['anchor_age', 'anchor_year']) # drop anchor_age and anchor_year and only keep admission_age
    return ans


In [10]:
merged_df = merge_df(df_chef=ed_edstays_df, df_2=hosp_patients_df, df_3=hosp_admissions_df)

In [21]:
# merged_df.head(15)
merged_df['language'].value_counts()

language
English                   286659
Spanish                    13579
Russian                     5135
Chinese                     4438
Kabuverdianu                3941
Haitian                     1555
Portuguese                  1414
Other                        719
Vietnamese                   565
American Sign Language       550
Modern Greek (1453-)         549
Italian                      523
Arabic                       412
Persian                      372
Polish                       260
Amharic                      202
Korean                       195
Thai                         192
Somali                       167
French                       133
Khmer                         96
Bengali                       95
Japanese                      89
Hindi                         84
Armenian                      50
Name: count, dtype: int64

In [12]:
cols = ['admission_age']
for col in cols:
    print(col, merged_df[col].isnull().sum() / merged_df.shape[0]) # check the percentage of missing values

admission_age 0.00017878693067536764


In [13]:
# merged_df[merged_df['admission_age'].isnull()] # check the rows with missing values

In [14]:
new_merged_df = pd.merge(ed_edstays_df, hosp_admissions_df, on='hadm_id', how='left')
ed_edstays_df['hadm_id'].isnull().sum() / ed_edstays_df.shape[0] # check the percentage of missing values

np.float64(0.5224130589738101)

def merge(df_chef, df_2, df_3):
    # df_1 contains subject_id, df_2 contains age, df_3 contains insurance, languge, 